In [8]:
#Libraries
library(glmnet)

# Code for submission test with lasso regularization

In [38]:
#Training data initialiation
train <- read.csv(file = '../../Data/training_data.csv')
train <- na.omit(train)
train.regr <- train[,-3]
train.regr$Intensity <- as.numeric(train.regr$Intensity)

#A DECOMENTER POUR LA SUBMISSION
#y.train <- train.regr$"VALENCE.PLEASANTNESS"
#train.regr <- train.regr[,-2]
#x.train <- as.matrix(train.regr)

#Test data initialisation
test <- read.csv(file = '../../Data/test_data.csv')
test <- na.omit(test)
test <- test[,-1]
test$Intensity <- as.numeric(test$Intensity)
x.test <- as.matrix(test)

#Validation set temporaraire le temps d'optimiser 2-3 hyperparametres A COMMENTER POUR LA SUBMISSION
idx.train <- sample(nrow(train.regr), nrow(train.regr)*1/2)
train.regr <- train.regr[idx.train,]
validation <- train.regr[-idx.train,]

y.train <- train.regr$"VALENCE.PLEASANTNESS"
train.regr <- train.regr[,-2]
x.train <- as.matrix(train.regr)

y.val <- validation$"VALENCE.PLEASANTNESS"
validation <- validation[,-2]
x.val <- as.matrix(validation)


In [39]:
#Pas utilisé pour le moment, à tester aussi
##Declaring a function to remove predictors with 0 variance. This function returns the indices of the predicors with 0 variance.
#ZeroVar <- function(data) {
#    out <- lapply(data, function(x) length(unique(x)))
#    want <- which(!out > 1)
#    unlist(want)
#}
#
##Applying the ZeroVar function
#train.zeroVar <- train.regr[,-ZeroVar(train.regr)]

In [40]:
#Finding best lambda value through cross-validation
L2.cv <- cv.glmnet(x.train, y.train, alpha = 1, nfold = 10)
lambda_best <- L2.cv$lambda.min 
lambda_best


[1] 1.377697

In [41]:
#Training the model with the best lambda (1.38727064964386)
Lasso_model <- glmnet(x.train, y.train, alpha = 1, lambda = lambda_best)

summary(Lasso_model)

#rmse avec les validations splits
sqrt(mean((y.val - predict(Lasso_model, x.val))^2))

          Length Class     Mode   
a0           1   -none-    numeric
beta      4870   dgCMatrix S4     
df           1   -none-    numeric
dim          2   -none-    numeric
lambda       1   -none-    numeric
dev.ratio    1   -none-    numeric
nulldev      1   -none-    numeric
npasses      1   -none-    numeric
jerr         1   -none-    numeric
offset       1   -none-    logical
call         5   -none-    call   
nobs         1   -none-    numeric

[1] 18.73195

In [42]:
result <- predict(Lasso_model,x.test)

In [85]:
id <- 1:68
final.data <- data.frame(Id = id,VALENCE.PLEASANTNESS = result)
write.csv(final.data,"../../Submission/Lasso.csv",row.names = FALSE)